In [22]:
from selenium import webdriver

from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
from selenium.common.exceptions import NoSuchElementException

import chromedriver_autoinstaller

import time

import pandas as pd
import json

import traceback

chromedriver_autoinstaller.install()

'/opt/anaconda3/envs/mkwenv/lib/python3.10/site-packages/chromedriver_autoinstaller/107/chromedriver'

In [24]:
with open('./json/KAKAO_INFO.json','r') as file:
    KAKAO_DATA = json.load(file)
SCROLL_PAUSE_TIME = 3

In [25]:
%%time
for code, html_info in KAKAO_DATA.items():
    try:
        driver = webdriver.Chrome()
        driver.get(html_info['url'])
        last_height = driver.execute_script("return document.body.scrollHeight")

        while True:
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(SCROLL_PAUSE_TIME)
            new_height = driver.execute_script("return document.body.scrollHeight")
            if new_height == last_height:
                print(f'link:{code},height:{last_height}')
                break
            last_height = new_height

        name_list = driver.find_elements(By.XPATH, html_info['name'])
        price_list = driver.find_elements(By.XPATH, html_info['price'])
        if len(price_list)!=len(name_list):
            raise Exception(f'List Length for {code} not match. name:{len(name_list)},price:{len(price_list)}')
            
        total_list = list(zip(name_list,price_list))    
        info_df = pd.DataFrame([{'name':x[0].text,
                                 'price':x[1].text,
                                 'type':code} for x in total_list])

        info_df.to_csv(f'./csv/KAKAO_{code}.csv')
        driver.quit()
    except Exception as e:
        traceback.print_exc()
        driver.quit()
    
    

link:birthday,height:23892
link:samll_gift,height:35507
link:health,height:24251
link:small_luxury,height:24373
link:luxury,height:33872
link:baby,height:14103
link:touched,height:23782
link:funny,height:33631
CPU times: user 2.84 s, sys: 292 ms, total: 3.13 s
Wall time: 5min 55s
